In [1]:
from zenml import __version__
print(f"ZenML version: {__version__}")


ZenML version: 0.70.0


In [4]:
from pymongo import MongoClient

client = MongoClient('mongodb://rag_mongodb:27017/')
db = client['rag_db']
collection = db['test_collection']

# Insert a test document
test_doc = {'message': 'Hello, MongoDB!'}
collection.insert_one(test_doc)

# Retrieve the document
retrieved_doc = collection.find_one({'message': 'Hello, MongoDB!'})

print(retrieved_doc)

{'_id': ObjectId('674e26a2763cbc2645753195'), 'message': 'Hello, MongoDB!'}


In [5]:
from qdrant_client import QdrantClient

client = QdrantClient(host='rag_qdrant', port=6333)

# Create a collection
client.recreate_collection(
    collection_name='test_collection',
    vectors_config={'size': 3, 'distance': 'Cosine'}
)

if client.collection_exists("test_collection"):
    client.delete_collection("documents")

client.create_collection(
    collection_name="documents",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE),
)

# Upload a vector
vector = [0.1, 0.2, 0.3]
payload = {'message': 'Hello, Qdrant!'}

client.upsert(
    collection_name='test_collection',
    points=[
        {
            'id': 1,
            'vector': vector,
            'payload': payload,
        }
    ]
)

# Search for the vector
results = client.search(
    collection_name='test_collection',
    query_vector=vector,
    limit=1
)

print(results)


HTTP Request: DELETE http://rag_qdrant:6333/collections/test_collection "HTTP/1.1 200 OK"


/tmp/ipykernel_39/1614699000.py:6: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


HTTP Request: PUT http://rag_qdrant:6333/collections/test_collection "HTTP/1.1 200 OK"
HTTP Request: PUT http://rag_qdrant:6333/collections/test_collection/points?wait=true "HTTP/1.1 200 OK"
HTTP Request: POST http://rag_qdrant:6333/collections/test_collection/points/search "HTTP/1.1 200 OK"
[ScoredPoint(id=1, version=0, score=0.9999998, payload={'message': 'Hello, Qdrant!'}, vector=None, shard_key=None, order_value=None)]
